In [1]:
# !pip install tfrecord matplotlib imutils tqdm ipywidgets

In [1]:
import torch
from apex import amp
from torchvision import models
from tqdm.notebook import tqdm
from cars196 import Cars196
from resnet50 import Resnet50
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
data_dir = '/root/data/cars196/orig/'

transforms = T.Compose([T.Resize(224), 
                        T.RandomCrop((224, 224)),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])])
train_dataset = Cars196(data_dir, train=True, transform=transforms, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=0)

In [4]:
model = models.resnet50().cuda()
model.fc = torch.nn.Linear(2048, train_dataset.num_classes).cuda()
criterion = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.01,
                                momentum=0.9,
                                weight_decay=1e-4)
model, optimizer = amp.initialize(model, optimizer)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [5]:
images, target  = next(iter(train_loader))
images = images.cuda(non_blocking=True)
target = target.cuda(non_blocking=True)
output = model(images)

[2021-01-27 06:39:10.312 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-594def216eaae0b31fbf025840e5:6460 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-01-27 06:39:10.479 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-594def216eaae0b31fbf025840e5:6460 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [16]:
for i, (images, target) in enumerate(train_loader):
    images = images.cuda(non_blocking=True)
    target = target.cuda(non_blocking=True)
    output = model(images)
    loss = criterion(output, target)
    optimizer.zero_grad()
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    optimizer.step()
    print(loss)

'for i, (images, target) in enumerate(train_loader):\n    images = images.cuda(non_blocking=True)\n    target = target.cuda(non_blocking=True)\n    output = model(images)\n    loss = criterion(output, target)\n    optimizer.zero_grad()\n    with amp.scale_loss(loss, optimizer) as scaled_loss:\n        scaled_loss.backward()\n    optimizer.step()\n    print(loss)'